In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
#from numpy import genfromtxt


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

from keras.models import model_from_json
import cv2, os
from keras.layers import Flatten
from keras.utils.vis_utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [18]:
train_dir = r"dataset/ImageEmotion (85-15)/train"
test_dir = r"dataset/ImageEmotion (85-15)/test"

SEED = 12
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
momentum = 0.9
EPOCHS = 500
#FINE_TUNING_EPOCHS = 30
LR = 0.001
NUM_CLASSES = 8
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Amusement', 'Anger', 'Awe', 'Contentment', 'Disgust', 'Excitement', 'Fear', 'Sadness']
CLASS_LABELS_EMOJIS = ["🥳", "😡", "😯", "😌", "🤢" ,"🤩", "😱" , "😔" ]

In [4]:
train_imagedata = np.load(r'FeatureFiles/train_imagedata_unscaled.npy', mmap_mode='r')
train_labels = np.load(r'FeatureFiles/train_labels.npy', mmap_mode='r')
test_imagedata = np.load(r'FeatureFiles/test_imagedata_unscaled.npy', mmap_mode='r')
test_labels = np.load(r'FeatureFiles/test_labels.npy', mmap_mode='r')

In [6]:
data_augmentation = tf.keras.Sequential([
                            tf.keras.layers.RandomFlip('horizontal'),
                            tf.keras.layers.RandomRotation(1./36),
                            tf.keras.layers.RandomHeight(0.05),
                            tf.keras.layers.RandomWidth(0.1)
                            ])

class Custom_DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, data, labels, batch_size, shuffle, augmentation= None):
        self.batch_size = batch_size
        self.data = data
        self.labels = labels
        self.indices = [i for i in range(len(data))]
        self.shuffle = shuffle
        self.augmentation = augmentation
        self.on_epoch_end()
        
    def __len__(self):
        if len(self.indices) % self.batch_size:
            return len(self.indices) // self.batch_size + 1
        else:
            return len(self.indices) // self.batch_size
    
    def on_epoch_end(self):
        self.index = np.arange(len(self.indices))
        if self.shuffle == True:
            np.random.shuffle(self.index)
    
    def __getitem__(self, index):
        index = self.index[index * self.batch_size:(index+1) * self.batch_size]
        batch = [self.indices[k] for k in index]
        
        X, y = self.__get_data(batch)
        return X, y
        
    def __get_data(self, batch):
        X = np.zeros(shape=(self.batch_size, 224,224,3))
        y = np.zeros(shape=(self.batch_size, 8))
        
        for i,ID in enumerate(batch):
            X[i] = self.data[ID]
            y[i] = self.labels[ID]
            
        if len(batch) != self.batch_size:
            req_num = self.batch_size - len(batch)
            X[len(batch):] = self.data[:req_num]
            y[len(batch):] = self.labels[:req_num]
        
        if self.augmentation:
            return self.augmentation(X), y
        else:
            return X, y

In [7]:
train_generator = Custom_DataGenerator(train_imagedata, train_labels, 64, True, data_augmentation)
test_generator = Custom_DataGenerator(test_imagedata, test_labels, 64, False)

In [19]:
# def get_pixel(img, center, x, y):
    
#     new_value = 0
    
#     try:
#         # If local neighbourhood pixel
#         # value is greater than or equal
#         # to center pixel values then
#         # set it to 1
#         if img[x][y] >= center:
#             new_value = 1
            
#     except:
#         # Exception is required when
#         # neighbourhood value of a center
#         # pixel value is null i.e. values
#         # present at boundaries.
#         pass
    
#     return new_value

# # Function for calculating LBP
# def lbp_calculated_pixel(img, x, y):

# 	center = img[x][y]

# 	val_ar = []
	
# 	# top_left
# 	val_ar.append(get_pixel(img, center, x-1, y-1))
	
# 	# top
# 	val_ar.append(get_pixel(img, center, x-1, y))
	
# 	# top_right
# 	val_ar.append(get_pixel(img, center, x-1, y + 1))
	
# 	# right
# 	val_ar.append(get_pixel(img, center, x, y + 1))
	
# 	# bottom_right
# 	val_ar.append(get_pixel(img, center, x + 1, y + 1))
	
# 	# bottom
# 	val_ar.append(get_pixel(img, center, x + 1, y))
	
# 	# bottom_left
# 	val_ar.append(get_pixel(img, center, x + 1, y-1))
	
# 	# left
# 	val_ar.append(get_pixel(img, center, x, y-1))
	
# 	# Now, we need to convert binary
# 	# values to decimal
# 	power_val = [1, 2, 4, 8, 16, 32, 64, 128]

# 	val = 0
	
# 	for i in range(len(val_ar)):
# 		val += val_ar[i] * power_val[i]
		
# 	return val,val_ar

# def normalize_nparray(np_arr):
#     denom = 0
#     for i in range(len(np_arr)):
#         denom += np_arr[i]**2
#     denom = denom**0.5
#     for i in range(len(np_arr)):
#         np_arr[i] = np_arr[i]/denom
    
#     return np_arr

# def get_lbp(sample, img_gray):
#     height, width, _ = img_gray.shape
#     img_lbp = np.zeros((height, width),
#                     np.uint8)
#     arr_hist=np.empty([8])
#     count=np.zeros([59])
#     for i in range(0, height):
#         for j in range(0, width):
#             img_lbp[i, j],arr_hist = lbp_calculated_pixel(img_gray, i, j)
#             for k in range(58):
#                 if np.array_equal(sample[k,], arr_hist):
#                     count[k]+=1
#     temp=0
#     for i in range(59):
#         temp=count[i]+temp
#     count[58]=(224*224)-temp-count[58]
    
#     txtr_mean = np.mean(img_lbp)
#     count = normalize_nparray(count)
    
#     return txtr_mean, count

# def lbp_extractor(sample, inputs):
#     mean_txtr_per_batch = np.zeros(shape=(BATCH_SIZE, 1))
#     inputs_gray = tf.image.rgb_to_grayscale(inputs)
    
#     i = 0
#     while i < BATCH_SIZE:
#         img_gray = inputs_gray[i]
#         mean_txtr_per_img, hist_per_img = get_lbp(sample, img_gray)
#         mean_txtr_per_batch[i] = mean_txtr_per_img
#         if i == 0:
#             hist_per_batch = hist_per_img
#         else:
#             hist_per_batch = np.concatenate([hist_per_batch, hist_per_img], axis=0)
#         i += 1
#         print(f'Done for {i} images')
    
#     lbp_features_per_batch = np.concatenate([mean_txtr_per_batch, hist_per_batch], axis=1)
    
#     return lbp_features_per_batch

In [8]:
""" Returns mean value of RGB """
def mean(inputs):
    # flatten image to be 2D and compute mean rgb
    mean_rgb_val = mean_helper(inputs)
    # convert image to hsv scale
    #hsv = cv2.cvtColor(img_data, cv2.COLOR_RGB2HSV)
    hsv = tf.image.rgb_to_hsv(inputs)
    # calculate mean
    mean_hsv_val = mean_helper(hsv)
    return mean_rgb_val, mean_hsv_val

""" Calculates mean value of a plane given a 3D matrix """
def mean_helper(org_mat):
    # "flatten" matrix to a 2D matrix
    temp = org_mat
    temp = tf.math.reduce_mean(temp,axis=1)
    temp = tf.math.reduce_mean(temp,axis=1)
    mean_val = temp
    return mean_val

""" Calculates pleasure, arousal, dominance values of the image"""
def calculate_pad(hsv):
    batch=hsv.shape[0]
    i = 0
    while i < BATCH_SIZE:
    #for i in range(BATCH_SIZE):
        saturation = hsv[i][1]
        brightness = hsv[i][2] # or 'value' in hsv
        pleasure = 0.69 * brightness + 0.22 * saturation
        arousal = 0.31 * brightness + 0.6 * saturation
        dominance = 0.76 * brightness + 0.32 * saturation
        pleasure = tf.reshape(pleasure, (1,1))
        arousal = tf.reshape(arousal, (1,1))
        dominance = tf.reshape(dominance, (1,1))
        pad_per_image = tf.concat([pleasure, arousal, dominance],axis= 1)
        pad_per_image = tf.reshape(pad_per_image, (1,1,3))
        if i == 0:
            pad = pad_per_image
        else:
            pad = tf.concat([pad, pad_per_image], axis= 0)
        i += 1
    return pad

def lowfeature_extractor(inputs):
    rgb, hsv = mean(inputs)
    rgb = Flatten()(rgb)
    pad = Flatten()(calculate_pad(hsv))
    hsv = Flatten()(hsv)
    #txtr = lbp_extractor(inputs)
    low_level_features = tf.concat([rgb, pad, hsv], axis= 1)
    return low_level_features

In [11]:
basee_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(224,224,3), include_top=False,
                                                weights= 'imagenet')

In [15]:
preprocess_fn = tf.keras.applications.inception_v3.preprocess_input

# mainobject_model = tf.keras.applications.resnet.ResNet101(input_shape=(224,224,3), include_top=False, 
#                                                              weights='imagenet')

def base_model_feature_extractor(inputs):
    y = basee_model(inputs)
    y = tf.keras.layers.GlobalAveragePooling2D()(y)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(y)
    
    return x, y

# def mainobject_model_feature_extractor(inputs):
#     y = mainobject_model(inputs)
#     y = tf.keras.layers.GlobalAveragePooling2D()(y)
    
#     return y

def classifier(features):
#     x = tf.keras.layers.Dense(1500, activation='relu', 
#                               kernel_regularizer= tf.keras.regularizers.l2(0.001))(features)
#     x = tf.keras.layers.Dense(500, activation='relu', 
#                               kernel_regularizer= tf.keras.regularizers.l2(0.01))(x)
#     x = tf.keras.layers.Dense(100, activation='relu', 
#                               kernel_regularizer= tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', 
                              #kernel_regularizer= tf.keras.regularizers.l2(0.01),
                              name='classification')(features)
    
    return x
        
def final_model(inputs, base_model, mainobject_model):
    inputs_copy1 = tf.tile(inputs, [1,1,1,1])
    inputs_copy1 = preprocess_fn(inputs_copy1)
    inputs_copy1 = inputs_copy1 * 1./255
    
    inputs_copy2 = tf.tile(inputs, [1,1,1,1])
    inputs_copy2 = inputs_copy2 * 1./255
    
    x, CNN_features = base_model(inputs_copy1)
    MainObject_features = mainobject_model(tf.reshape(inputs_copy1, (299,299,3)))
    lowfeatures = lowfeature_extractor(inputs_copy2)
    
    merged_features = tf.concat([MainObject_features, CNN_features, lowfeatures], axis=1)
    classification_output = classifier(merged_features)
    
    return classification_output

def define_compile_model(base_weights):
    
    #inputs1 = tf.keras.layers.Input(shape=(150,))
    inputs = tf.keras.layers.Input(shape=(224,224,3))
    base_model = tf.keras.Model(inputs= inputs, outputs= base_model_feature_extractor(inputs))
    base_model.load_weights(base_weights)
    mainobject_model = tf.keras.applications.inception_v3.InceptionV3(include_top=True, 
                                                             weights='imagenet')
    mainobject_model.trainable = False
    #base_model.layers[-1].trainable = False
    classification_output = final_model(inputs, base_model, mainobject_model) 
    model = tf.keras.Model(inputs= inputs, outputs= classification_output)
    model.compile(optimizer=tf.keras.optimizers.SGD(0.001), 
                loss='categorical_crossentropy',
                metrics = ['acc'])
  
    return model

In [16]:
base_weights = r'h5 files/InceptionV3-022-0.885500-0.624046.h5'
model = define_compile_model(base_weights)
#model.load_weights(r'Idea2_ResNet101-007-0.725108-0.641226.h5')

model.summary()

ValueError: Exception encountered when calling layer "tf.reshape" (type TFOpLambda).

Dimension size must be evenly divisible by 150528 but is 268203 for '{{node tf.reshape/Reshape}} = Reshape[T=DT_FLOAT, Tshape=DT_INT32](Placeholder, tf.reshape/Reshape/shape)' with input shapes: [?,224,224,3], [3] and with input tensors computed as partial shapes: input[1] = [299,299,3].

Call arguments received by layer "tf.reshape" (type TFOpLambda):
  • tensor=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • shape=('299', '299', '3')
  • name=None

In [11]:
#for plotting model
import keras.utils.vis_utils
from importlib import reload
reload(keras.utils.vis_utils)


from keras.utils.vis_utils import plot_model    
plot_model(model, to_file='model_i1.png', show_shapes=True, show_layer_names=True)

dot: graph is too large for cairo-renderer bitmaps. Scaling by 0.230906 to fit



In [26]:
checkpoint = ModelCheckpoint('Idea2_ResNet50-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1, 
                        monitor='val_acc',save_best_only=True, mode='auto') 
#earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto')

In [27]:
model.fit(train_generator,
        batch_size = BATCH_SIZE,
        epochs= 100,
        validation_data = test_generator,
        validation_batch_size= BATCH_SIZE,
        callbacks= [checkpoint]
        )
#     score = model.evaluate((test_objectdetails,test_image_data,test_lowfeatures), test_labels, verbose=0)
#     print('val_acc: ', round(score[0],4), ' ', 'val_loss: ', round(score[1],4))
#history = pd.DataFrame(history.history)

Epoch 1/100
290/290 [==============================] - ETA: 0s - loss: 1.4373 - acc: 0.4934
Epoch 1: val_acc improved from -inf to 0.52734, saving model to Idea2_ResNet50-001-0.493427-0.527344.h5
290/290 [==============================] - 1339s 5s/step - loss: 1.4373 - acc: 0.4934 - val_loss: 1.3588 - val_acc: 0.5273
Epoch 2/100
290/290 [==============================] - ETA: 0s - loss: 1.0320 - acc: 0.6421
Epoch 2: val_acc improved from 0.52734 to 0.57181, saving model to Idea2_ResNet50-002-0.642080-0.571815.h5
290/290 [==============================] - 1323s 5s/step - loss: 1.0320 - acc: 0.6421 - val_loss: 1.2525 - val_acc: 0.5718
Epoch 3/100
290/290 [==============================] - ETA: 0s - loss: 0.9244 - acc: 0.6785
Epoch 3: val_acc improved from 0.57181 to 0.58293, saving model to Idea2_ResNet50-003-0.678502-0.582933.h5
290/290 [==============================] - 1331s 5s/step - loss: 0.9244 - acc: 0.6785 - val_loss: 1.2291 - val_acc: 0.5829
Epoch 4/100
290/290 [================

KeyboardInterrupt: 